In [1]:
proj_path='/content/drive/MyDrive/S4 - 16824 VLR/generative-modeling/gan'

### Git Stuff

### Colab stuff

In [4]:
# one-time data set-up
# #!pip show "clean-fid"
# !cd "$proj_path/$hw_dir" && source "setup_data.sh"
# !ls "/usr/local/lib/python3.7/dist-packages/cleanfid"
# !mkdir "/usr/local/lib/python3.7/dist-packages/cleanfid/stats"
# !cd "$proj_path/$hw_dir/$tasks" && cp cub_clean_custom_na.npz "/usr/local/lib/python3.7/dist-packages/cleanfid/stats/cub_clean_custom_na.npz"

## Notebook

In [6]:
import sys, pathlib
sys.path.append('./gan')
sys.path.append('./vae')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### GAN

In [7]:
from gan.networks import *
from gan.train import *
from gan.q1_4 import compute_discriminator_loss, compute_generator_loss
from networks import Discriminator, Generator

In [ ]:
gen = Generator().cuda().to(memory_format=torch.channels_last)
disc = Discriminator().cuda().to(memory_format=torch.channels_last)
prefix = "data_gan/"
os.makedirs(prefix, exist_ok=True)
# TODO 1.3.2: Run this line of code.
train_model(
    gen,
    disc,
    num_iterations=int(3e4),
    batch_size=16,
    prefix=prefix,
    gen_loss_fn=compute_generator_loss,
    disc_loss_fn=compute_discriminator_loss,
    log_period=1000,
    wandb_logging=True
)

data dir exist:True
Epoch after iters:0
Epoch after iters:737
compute FID of a model with cub-32 statistics


FID model: 100%|████████████████████████████████████████| 625/625 [04:16<00:00,  2.43it/s]


Iteration 1000 FID: 429.1199568290983
Epoch after iters:1474
compute FID of a model with cub-32 statistics


FID model: 100%|████████████████████████████████████████| 625/625 [04:12<00:00,  2.48it/s]


Iteration 2000 FID: 366.0120388735245
Epoch after iters:2211
Epoch after iters:2948
compute FID of a model with cub-32 statistics


...

FID model: 100%|████████████████████████████████████████| 625/625 [04:11<00:00,  2.48it/s]


Iteration 14000 FID: 297.22916539430446
Epoch after iters:14003
Epoch after iters:14740
compute FID of a model with cub-32 statistics


FID model: 100%|████████████████████████████████████████| 625/625 [06:13<00:00,  1.67it/s]


Iteration 15000 FID: 288.2964057096409
Epoch after iters:15477
compute FID of a model with cub-32 statistics


FID model:   0%|                                                  | 0/625 [00:00<?, ?it/s]

### VAE

In [ ]:
from vae.model import *
from vae.train import *
#from vae.utils import compute_discriminator_loss, compute_generator_loss
#from networks import Discriminator, Generator